# Generic Chrome WebDriver

This notebook demonstrates using reusable utilities for Chrome WebDriver automation.

## Configuration

All settings are loaded from the `.env` file:
- **Chrome settings**: Headless mode, window size, implicit wait
- **Directories**: Downloads, Chrome profile, log file
- **Microsoft credentials**: Username and password
- **Concurrency**: Max workers for parallel execution

Create a `.env` file from `.env.example` and customize your settings.

## Quick Start

1. Run the first two cells to import utilities and display configuration
2. Create a driver instance (configured from .env)
3. Use the driver for automation tasks
4. Try the Microsoft login or concurrency examples

In [ ]:
# Import utilities
from Utilities.driver_factory import create_driver
from Utilities.config import Config, print_config
from Utilities.logging_setup import setup_logging
from Utilities.microsoft_login import microsoft_login, wait_for_login_redirect

# Selenium imports (for custom scripts)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Setup logging and display configuration
setup_logging()
print_config()

# Single Driver Example

Single driver, good for linear scripts and testing

In [ ]:
# Create driver using configuration from .env
driver = create_driver(
    headless=Config.HEADLESS,
    profile_dir=Config.get_profile_dir(),
    download_dir=Config.DOWNLOAD_DIR,
    implicit_wait=Config.IMPLICIT_WAIT
)

# Concurrency Example

Example of concurrency using multiple drivers and a worker pool

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import logging

def worker_search(worker_id, search_query):
    """
    Worker function that creates a driver, performs a Google search, and closes.
    
    Args:
        worker_id: Unique identifier for this worker
        search_query: Query to search on Google
        
    Returns:
        dict: Results of the operation
    """
    worker_driver = None
    try:
        logging.info(f"Worker {worker_id}: Starting...")
        
        # Create a new driver for this worker using config settings
        worker_driver = create_driver(
            headless=Config.HEADLESS,
            implicit_wait=Config.IMPLICIT_WAIT
        )
        
        # Navigate to Google
        logging.info(f"Worker {worker_id}: Navigating to Google...")
        worker_driver.get("https://www.google.com")
        
        # Wait for and find the search box
        search_box = WebDriverWait(worker_driver, 10).until(
            EC.presence_of_element_located((By.NAME, "q"))
        )
        
        # Perform search
        logging.info(f"Worker {worker_id}: Searching for '{search_query}'...")
        search_box.send_keys(search_query)
        search_box.submit()
        
        # Wait for results to load
        WebDriverWait(worker_driver, 10).until(
            EC.presence_of_element_located((By.ID, "search"))
        )
        
        # Get the page title
        page_title = worker_driver.title
        logging.info(f"Worker {worker_id}: Search completed. Page title: {page_title}")
        
        # Simulate some work
        time.sleep(2)
        
        return {
            "worker_id": worker_id,
            "query": search_query,
            "title": page_title,
            "success": True
        }
        
    except Exception as e:
        logging.error(f"Worker {worker_id}: Error occurred - {str(e)}")
        return {
            "worker_id": worker_id,
            "query": search_query,
            "success": False,
            "error": str(e)
        }
        
    finally:
        # Always close the driver
        if worker_driver:
            logging.info(f"Worker {worker_id}: Closing driver...")
            worker_driver.quit()


# Search queries for workers
search_queries = [
    "Python Selenium tutorial",
    "Web scraping best practices",
    "Chrome WebDriver automation"
]

# Execute searches concurrently using config settings
logging.info(f"Starting {Config.MAX_WORKERS} concurrent workers...")
results = []

with ThreadPoolExecutor(max_workers=Config.MAX_WORKERS) as executor:
    # Submit all tasks
    futures = {
        executor.submit(worker_search, i+1, search_queries[i]): i 
        for i in range(min(Config.MAX_WORKERS, len(search_queries)))
    }
    
    # Collect results as they complete
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        
        if result["success"]:
            logging.info(f"✓ Worker {result['worker_id']} completed successfully")
        else:
            logging.error(f"✗ Worker {result['worker_id']} failed: {result.get('error', 'Unknown error')}")

# Summary
logging.info("\n" + "="*50)
logging.info("CONCURRENCY EXAMPLE COMPLETE")
logging.info("="*50)
successful = sum(1 for r in results if r["success"])
logging.info(f"Total workers: {len(results)}")
logging.info(f"Successful: {successful}")
logging.info(f"Failed: {len(results) - successful}")
logging.info("="*50)

# Microsoft Login Example

The `microsoft_login` utility automates Microsoft authentication:
1. Enter username and click "Next"
2. Enter password and click "Sign in"
3. Handle "Stay signed in?" prompt

It automatically loads credentials from the `.env` file.

In [ ]:
# Perform login using credentials from .env file
try:
    success = microsoft_login(
        driver,
        stay_signed_in=Config.STAY_SIGNED_IN,
        timeout=20
    )
    
    if success:
        logging.info("Microsoft login successful!")
        
        # Optional: Wait for redirect to specific page
        # wait_for_login_redirect(driver, expected_url="portal.office.com")
    else:
        logging.error("Microsoft login failed!")
        
except Exception as e:
    logging.error(f"Login error: {e}")

# Humanize Example

The `humanize` utility makes interactions look more human-like with:
- **Random typing speed** with occasional typos
- **Mouse movements** with natural curves
- **Click delays** and random pauses
- **Smooth scrolling** with variable speed
- **Form filling** with realistic timing

In [ ]:
from Utilities.humanize import (
    human_send_keys,
    human_click,
    human_scroll,
    human_pause,
    human_mouse_move,
    human_form_fill
)
import logging

# Example 1: Google search with humanized interactions
logging.info("=== Humanized Google Search Example ===")

# Navigate to Google
driver.get("https://www.google.com")
human_pause(1.0, 2.0)

# Find search box and type with human-like behavior
search_box = driver.find_element(By.NAME, "q")

# Human-like typing (with occasional typos and corrections)
human_send_keys(search_box, "Python Selenium automation")
human_pause(0.5, 1.0)

# Submit search with human click
search_box.submit()
human_pause(1.0, 2.0)

# Scroll down the results naturally
human_scroll(driver, direction="down", amount=300)
human_pause(0.5, 1.5)

# Scroll down more
human_scroll(driver, direction="down", amount=400)
human_pause(0.5, 1.0)

# Scroll back up
human_scroll(driver, direction="up", amount=200)
human_pause()

logging.info("=== Humanized interaction complete ===")


In [ ]:
# Example 2: Form filling with humanized interactions
logging.info("=== Humanized Form Fill Example ===")

# This example shows how to fill a form naturally
# Note: Replace with actual form elements from your target website

# Simulate form data (replace with real elements)
# form_data = {
#     "email": (email_field, "john.doe@example.com"),
#     "password": (password_field, "SecurePassword123"),
#     "remember_me": checkbox_element,  # Just click
#     "submit": submit_button  # Just click
# }
# 
# human_form_fill(driver, form_data)

logging.info("Form example (replace with real form elements)")
